## Framingham Reshaping

Turning the longitudinal Framingham data into a format suitable for multistate modeling.

We will focus on creating a data set suitable for four states: No disease, Hypertension, Cardiovascular disease (any), and Death (absorbing).

Some people will begin in State 2 due to PREVHYP.

Retained covariates will include: Age, Sex, BMI, Diabetes (y/n), and Smoker (y/n)

In [1]:
%matplotlib inline
import matplotlib
import pandas as pd
import numpy as np

In [2]:
framingham = pd.read_csv("Datasets/framingham.csv")

In [3]:
framingham

,SEX,RANDID,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,...,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
0,1,2448,195,39,106.0,70.0,0,0,26.97,0,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
1,1,2448,209,52,121.0,66.0,0,0,.,0,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
2,2,6238,250,46,121.0,81.0,0,0,28.73,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
3,2,6238,260,52,105.0,69.5,0,0,29.43,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
4,2,6238,237,58,108.0,66.0,0,0,28.5,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11622,1,9998212,173,46,126.0,82.0,0,0,19.17,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,0
11623,1,9998212,153,52,143.0,89.0,0,0,25.74,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,0
11624,2,9999312,196,39,133.0,86.0,1,30,20.91,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,4201
11625,2,9999312,240,46,138.0,79.0,1,20,26.39,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,4201


In [4]:
framingham.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11627 entries, 0 to 11626
Data columns (total 38 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEX       11627 non-null  int64  
 1   RANDID    11627 non-null  int64  
 2   TOTCHOL   11627 non-null  object 
 3   AGE       11627 non-null  int64  
 4   SYSBP     11627 non-null  float64
 5   DIABP     11627 non-null  float64
 6   CURSMOKE  11627 non-null  int64  
 7   CIGPDAY   11627 non-null  object 
 8   BMI       11627 non-null  object 
 9   DIABETES  11627 non-null  int64  
 10  BPMEDS    11627 non-null  object 
 11  HEARTRTE  11627 non-null  object 
 12  GLUCOSE   11627 non-null  object 
 13  PREVCHD   11627 non-null  int64  
 14  PREVAP    11627 non-null  int64  
 15  PREVMI    11627 non-null  int64  
 16  PREVSTRK  11627 non-null  int64  
 17  PREVHYP   11627 non-null  int64  
 18  TIME      11627 non-null  int64  
 19  PERIOD    11627 non-null  int64  
 20  HDLC      11627 non-null  ob

## Define the at risk population
We want to begin with people who do not already have cardiovascular disease

In [5]:
at_risk = framingham[(framingham["PREVAP"] == 0) & (framingham["PREVCHD"] == 0) & (framingham["PREVMI"] == 0) & (framingham["PREVSTRK"] == 0)] 
at_risk = at_risk[["RANDID", "TIME", "PERIOD", "TIMECVD", "TIMEDTH", "TIMEHYP", "PREVHYP", "AGE", "SEX", "BMI", "CURSMOKE", "DIABETES"]]
at_risk

,RANDID,TIME,PERIOD,TIMECVD,TIMEDTH,TIMEHYP,PREVHYP,AGE,SEX,BMI,CURSMOKE,DIABETES
0,2448,0,1,6438,8766,8766,0,39,1,26.97,0,0
1,2448,4628,3,6438,8766,8766,0,52,1,.,0,0
2,6238,0,1,8766,8766,8766,0,46,2,28.73,0,0
3,6238,2156,2,8766,8766,8766,0,52,2,29.43,0,0
4,6238,4344,3,8766,8766,8766,0,58,2,28.5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11622,9998212,2333,2,8766,8766,0,1,46,1,19.17,0,0
11623,9998212,4538,3,8766,8766,0,1,52,1,25.74,0,0
11624,9999312,0,1,8766,8766,4201,0,39,2,20.91,1,0
11625,9999312,2390,2,8766,8766,4201,0,46,2,26.39,1,0


## Discover Ns

The last day of the Framingham study was day # 8,766.

Patients who reached day 8,766 **without** getting Hypertension would have TIMEHYP set to this number - meaning they could have gotten Hypertension after this day, but nobody would know about it.

In other words, if TIME in days is set to 8,766 for a category, it indicates the patient reached the end of the study period (or was censored) without that disease and/or without dying.

So we want to figure out how many unique individuals in the study **did** get diagnosed with Hypertension (State 2), get diagnosed with CVD (State 3), or Died before day 8,766 (State 4).

In [6]:
last_day_of_study = 8766

ind_ids = at_risk.drop_duplicates(subset=["RANDID"])["RANDID"].tolist()
died_ids = at_risk[(at_risk["TIME"]>0) & (at_risk["TIMEDTH"]<last_day_of_study)]["RANDID"].drop_duplicates()
cvd_ids = at_risk[(at_risk["TIME"]>0) & (at_risk["TIMECVD"]<last_day_of_study)]["RANDID"].drop_duplicates()
htn_ids = at_risk[((at_risk["TIME"]>0) & (at_risk["TIMEHYP"]<last_day_of_study)) | (at_risk["PREVHYP"]==1)]["RANDID"].drop_duplicates()

There are 4,215 unique individuals in the study.

In [7]:
len(ind_ids)

4215

3,161 of them (75%) either started the study with Hypertension, or became Hypertensive after TIME 0 (entered State 2)

In [8]:
htn_ids

7          10552
10         11252
12         11263
18         12806
20         14367
          ...   
11614    9990894
11617    9993179
11620    9995546
11621    9998212
11625    9999312
Name: RANDID, Length: 3161, dtype: int64

1,315 of them (31%) got Cardiovascular disease after TIME 0 (entered State 3)

In [9]:
cvd_ids

1           2448
8          10552
13         11263
33         23727
36         24721
          ...   
11605    9982118
11611    9989287
11614    9990894
11617    9993179
11620    9995546
Name: RANDID, Length: 1315, dtype: int64

And 992 of them (24%) died after TIME 0 but before the last day of the study (entered State 4)

In [10]:
died_ids

8          10552
33         23727
36         24721
56         43770
73         66472
          ...   
11599    9973350
11605    9982118
11611    9989287
11614    9990894
11617    9993179
Name: RANDID, Length: 992, dtype: int64

Now that we know how many people reached each state, we need to figure out what state they started in.

Some people started in State 1 - No Disease, while others started in State 2 - Hypertension.

Some even had recorded data at TIME 0 and then no recorded data until period 3 (TIME 45xxish).

Transition times from one state to another are in the TIMEHYP, TIMECVD and TIMEDTH columns, NOT in the TIME column.  The TIME column just records VISIT times for longitudinal study participants, so we only care about the TIME column for State 1 and maybe State 2 (for people with PREVHYP == 1 **at** their initial time).

In [11]:
start_times = at_risk.drop_duplicates(subset=["RANDID"], keep="first")
start_times

,RANDID,TIME,PERIOD,TIMECVD,TIMEDTH,TIMEHYP,PREVHYP,AGE,SEX,BMI,CURSMOKE,DIABETES
0,2448,0,1,6438,8766,8766,0,39,1,26.97,0,0
2,6238,0,1,8766,8766,8766,0,46,2,28.73,0,0
5,9428,0,1,8766,8766,8766,0,48,1,25.34,1,0
7,10552,0,1,2089,2956,0,1,61,2,28.58,1,0
9,11252,0,1,8766,8766,4285,0,46,2,23.1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11613,9990894,0,1,6433,6433,2219,0,48,2,22,1,0
11616,9993179,0,1,6729,6729,4396,0,44,2,19.16,1,0
11619,9995546,0,1,5209,8766,735,0,52,2,21.47,0,0
11621,9998212,0,1,8766,8766,0,1,40,1,25.6,0,0


Looks like all 4,215 participants started at TIME 0.

In [12]:
start_times["TIME"].value_counts()

0    4215
Name: TIME, dtype: int64

2,917 of them (69%) were healthy, and 1,298 (31%) already had Hypertension.

In [13]:
start_times["PREVHYP"].value_counts()

0    2917
1    1298
Name: PREVHYP, dtype: int64

In [14]:
framingham_ms1 = at_risk.drop_duplicates(subset=["RANDID"], keep="first").reset_index()
framingham_ms1["STATE"] = framingham_ms1["PERIOD"].copy()
framingham_ms1.loc[framingham_ms1["PREVHYP"]==1, "STATE"] = 2
framingham_ms1["DAYS"] = 0
framingham_ms1

,index,RANDID,TIME,PERIOD,TIMECVD,TIMEDTH,TIMEHYP,PREVHYP,AGE,SEX,BMI,CURSMOKE,DIABETES,STATE,DAYS
0,0,2448,0,1,6438,8766,8766,0,39,1,26.97,0,0,1,0
1,2,6238,0,1,8766,8766,8766,0,46,2,28.73,0,0,1,0
2,5,9428,0,1,8766,8766,8766,0,48,1,25.34,1,0,1,0
3,7,10552,0,1,2089,2956,0,1,61,2,28.58,1,0,2,0
4,9,11252,0,1,8766,8766,4285,0,46,2,23.1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4210,11613,9990894,0,1,6433,6433,2219,0,48,2,22,1,0,1,0
4211,11616,9993179,0,1,6729,6729,4396,0,44,2,19.16,1,0,1,0
4212,11619,9995546,0,1,5209,8766,735,0,52,2,21.47,0,0,1,0
4213,11621,9998212,0,1,8766,8766,0,1,40,1,25.6,0,0,2,0


Now we have the first States recorded for all the participants.

We need to add rows for those who also experienced State 2 after TIME 0, and those who experienced State 3 (Cardiovascular disease) and State 4 (Death).

## Things we need to account for:

Sometimes a person might get Hypertension & CVD on the same day.

Or, they might get CVD & Die on the same day.

Or, they might get Hypertension and Die on the same day.

Or, potentially, all THREE things might occur on the same day.

We have to resolve these conflicts, or the model won't run.

So, wherever a person is recorded as transitioning from State 1 to State 2 & 3 or 3 & 4, or 2, 3 & 4 on the same day, for example, this must be replaced by a transition ONLY to state 4.

In [16]:
framingham_ms2 = at_risk[(at_risk["TIME"]>0) 
                         & (at_risk["PREVHYP"]==0) 
                         & (at_risk["TIMEHYP"]<last_day_of_study)].drop_duplicates(subset=["RANDID"], keep="first").reset_index()
framingham_ms2["STATE"] = 2
framingham_ms2["DAYS"] = framingham_ms2["TIMEHYP"].copy()
framingham_ms2

,index,RANDID,TIME,PERIOD,TIMECVD,TIMEDTH,TIMEHYP,PREVHYP,AGE,SEX,BMI,CURSMOKE,DIABETES,STATE,DAYS
0,10,11252,2072,2,8766,8766,4285,0,51,2,23.48,1,0,2,4285
1,18,12806,2170,2,8766,8766,8679,0,51,2,22.19,1,0,2,8679
2,36,24721,2268,2,6411,6411,4408,0,46,2,21.24,1,0,2,4408
3,50,40435,2198,2,8766,8766,5933,0,48,2,21.77,1,0,2,5933
4,53,43522,2225,2,8766,8766,5226,0,49,2,34.55,0,0,2,5226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143,11591,9967157,2195,2,3273,7362,7362,0,64,1,24.86,0,0,2,7362
1144,11594,9968499,1999,2,4177,4177,4177,0,49,1,24.91,1,0,2,4177
1145,11611,9989287,2148,2,7746,7746,7746,0,57,1,17.65,1,0,2,7746
1146,11617,9993179,2226,2,6729,6729,4396,0,50,2,21.22,1,0,2,4396


In [20]:
framingham_ms_temp = pd.concat([framingham_ms1, framingham_ms2], ignore_index=True)
framingham_ms_temp = framingham_ms_temp.drop_duplicates(subset=["RANDID", "DAYS"], keep="last").reset_index()
framingham_ms_temp = framingham_ms_temp.drop(columns=["level_0", "index"], axis=1)
framingham_ms_temp


,RANDID,TIME,PERIOD,TIMECVD,TIMEDTH,TIMEHYP,PREVHYP,AGE,SEX,BMI,CURSMOKE,DIABETES,STATE,DAYS
0,2448,0,1,6438,8766,8766,0,39,1,26.97,0,0,1,0
1,6238,0,1,8766,8766,8766,0,46,2,28.73,0,0,1,0
2,9428,0,1,8766,8766,8766,0,48,1,25.34,1,0,1,0
3,10552,0,1,2089,2956,0,1,61,2,28.58,1,0,2,0
4,11252,0,1,8766,8766,4285,0,46,2,23.1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5358,9967157,2195,2,3273,7362,7362,0,64,1,24.86,0,0,2,7362
5359,9968499,1999,2,4177,4177,4177,0,49,1,24.91,1,0,2,4177
5360,9989287,2148,2,7746,7746,7746,0,57,1,17.65,1,0,2,7746
5361,9993179,2226,2,6729,6729,4396,0,50,2,21.22,1,0,2,4396


Adding more State 2 people didn't result in any duplicate Days values.

That makes sense, since we pre-screened those who already had Hypertension.

In [21]:
framingham_ms3 = at_risk[(at_risk["TIME"]>0) & (at_risk["TIMECVD"]<last_day_of_study)].drop_duplicates(subset=["RANDID"], keep="first").reset_index()
framingham_ms3["STATE"] = 3
framingham_ms3["DAYS"] = framingham_ms3["TIMECVD"].copy()
framingham_ms3

,index,RANDID,TIME,PERIOD,TIMECVD,TIMEDTH,TIMEHYP,PREVHYP,AGE,SEX,BMI,CURSMOKE,DIABETES,STATE,DAYS
0,1,2448,4628,3,6438,8766,8766,0,52,1,.,0,0,3,6438
1,8,10552,1977,2,2089,2956,0,1,67,2,30.18,1,0,3,2089
2,13,11263,2178,2,5719,8766,0,1,49,2,31.36,0,0,3,5719
3,33,23727,2287,2,5592,5592,0,1,47,2,27.98,0,0,3,5592
4,36,24721,2268,2,6411,6411,4408,0,46,2,21.24,1,0,3,6411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,11605,9982118,2253,2,8346,8457,0,1,64,1,25.56,0,0,3,8346
1311,11611,9989287,2148,2,7746,7746,7746,0,57,1,17.65,1,0,3,7746
1312,11614,9990894,2219,2,6433,6433,2219,1,54,2,21.64,1,0,3,6433
1313,11617,9993179,2226,2,6729,6729,4396,0,50,2,21.22,1,0,3,6729


In [22]:
framingham_ms_temp2 = pd.concat([framingham_ms_temp, framingham_ms3], ignore_index=True)
framingham_ms_temp2 = framingham_ms_temp2.drop_duplicates(subset=["RANDID", "DAYS"], keep="last").reset_index()
framingham_ms_temp2 = framingham_ms_temp2.drop(columns=["level_0", "index"], axis=1)
framingham_ms_temp2

,RANDID,TIME,PERIOD,TIMECVD,TIMEDTH,TIMEHYP,PREVHYP,AGE,SEX,BMI,CURSMOKE,DIABETES,STATE,DAYS
0,2448,0,1,6438,8766,8766,0,39,1,26.97,0,0,1,0
1,6238,0,1,8766,8766,8766,0,46,2,28.73,0,0,1,0
2,9428,0,1,8766,8766,8766,0,48,1,25.34,1,0,1,0
3,10552,0,1,2089,2956,0,1,61,2,28.58,1,0,2,0
4,11252,0,1,8766,8766,4285,0,46,2,23.1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6501,9982118,2253,2,8346,8457,0,1,64,1,25.56,0,0,3,8346
6502,9989287,2148,2,7746,7746,7746,0,57,1,17.65,1,0,3,7746
6503,9990894,2219,2,6433,6433,2219,1,54,2,21.64,1,0,3,6433
6504,9993179,2226,2,6729,6729,4396,0,50,2,21.22,1,0,3,6729


Now that we've added the State 3 people, however, we can see that there were duplicates. 

While the original # of persons who got CVD was 1,315, only 1,143 rows were added to the final dataframe.

This means 172 people got Hypertension and CVD on the same day.

In [23]:
framingham_ms4 = at_risk[(at_risk["TIME"]>0) & (at_risk["TIMEDTH"]<last_day_of_study)].drop_duplicates(subset=["RANDID"], keep="first").reset_index()
framingham_ms4["STATE"] = 4
framingham_ms4["DAYS"] = framingham_ms4["TIMEDTH"].copy()
framingham_ms4

,index,RANDID,TIME,PERIOD,TIMECVD,TIMEDTH,TIMEHYP,PREVHYP,AGE,SEX,BMI,CURSMOKE,DIABETES,STATE,DAYS
0,8,10552,1977,2,2089,2956,0,1,67,2,30.18,1,0,4,2956
1,33,23727,2287,2,5592,5592,0,1,47,2,27.98,0,0,4,5592
2,36,24721,2268,2,6411,6411,4408,0,46,2,21.24,1,0,4,6411
3,56,43770,2198,2,6384,6410,723,1,58,2,33.29,0,1,4,6410
4,73,66472,2174,2,8085,8423,1463,1,66,2,30.16,0,0,4,8423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,11599,9973350,2204,2,6562,6562,0,1,57,2,22.14,1,1,4,6562
988,11605,9982118,2253,2,8346,8457,0,1,64,1,25.56,0,0,4,8457
989,11611,9989287,2148,2,7746,7746,7746,0,57,1,17.65,1,0,4,7746
990,11614,9990894,2219,2,6433,6433,2219,1,54,2,21.64,1,0,4,6433


In [24]:
framingham_ms_temp3 = pd.concat([framingham_ms_temp2, framingham_ms4], ignore_index=True)
framingham_ms_temp3 = framingham_ms_temp3.drop_duplicates(subset=["RANDID", "DAYS"], keep="last").reset_index()
framingham_ms_temp3 = framingham_ms_temp3.drop(columns=["level_0", "index"], axis=1)
framingham_ms_temp3

,RANDID,TIME,PERIOD,TIMECVD,TIMEDTH,TIMEHYP,PREVHYP,AGE,SEX,BMI,CURSMOKE,DIABETES,STATE,DAYS
0,2448,0,1,6438,8766,8766,0,39,1,26.97,0,0,1,0
1,6238,0,1,8766,8766,8766,0,46,2,28.73,0,0,1,0
2,9428,0,1,8766,8766,8766,0,48,1,25.34,1,0,1,0
3,10552,0,1,2089,2956,0,1,61,2,28.58,1,0,2,0
4,11252,0,1,8766,8766,4285,0,46,2,23.1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6788,9973350,2204,2,6562,6562,0,1,57,2,22.14,1,1,4,6562
6789,9982118,2253,2,8346,8457,0,1,64,1,25.56,0,0,4,8457
6790,9989287,2148,2,7746,7746,7746,0,57,1,17.65,1,0,4,7746
6791,9990894,2219,2,6433,6433,2219,1,54,2,21.64,1,0,4,6433


As we might have expected, even **more** persons (287) died on the same day they recieved another diagnosis.

In [32]:
framingham_ms = framingham_ms_temp3.copy()
framingham_ms = framingham_ms.drop(columns=["TIME", "PERIOD", "TIMECVD", "TIMEDTH", "TIMEHYP", "PREVHYP"], axis=1)
framingham_ms["YEARS"] = framingham_ms["DAYS"]/365
framingham_ms

,RANDID,AGE,SEX,BMI,CURSMOKE,DIABETES,STATE,DAYS,YEARS
0,2448,39,1,26.97,0,0,1,0,0.000000
1,6238,46,2,28.73,0,0,1,0,0.000000
2,9428,48,1,25.34,1,0,1,0,0.000000
3,10552,61,2,28.58,1,0,2,0,0.000000
4,11252,46,2,23.1,1,0,1,0,0.000000
...,...,...,...,...,...,...,...,...,...
6788,9973350,57,2,22.14,1,1,4,6562,17.978082
6789,9982118,64,1,25.56,0,0,4,8457,23.169863
6790,9989287,57,1,17.65,1,0,4,7746,21.221918
6791,9990894,54,2,21.64,1,0,4,6433,17.624658


In [33]:
framingham_ms = framingham_ms.sort_values(by=['RANDID', 'YEARS'])
framingham_ms = framingham_ms.reset_index()
framingham_ms = framingham_ms.drop(columns=["index"], axis=1)
framingham_ms

,RANDID,AGE,SEX,BMI,CURSMOKE,DIABETES,STATE,DAYS,YEARS
0,2448,39,1,26.97,0,0,1,0,0.000000
1,2448,52,1,.,0,0,3,6438,17.638356
2,6238,46,2,28.73,0,0,1,0,0.000000
3,9428,48,1,25.34,1,0,1,0,0.000000
4,10552,61,2,28.58,1,0,2,0,0.000000
...,...,...,...,...,...,...,...,...,...
6788,9995546,52,2,21.47,0,0,1,0,0.000000
6789,9995546,58,2,22.55,0,0,3,5209,14.271233
6790,9998212,40,1,25.6,0,0,2,0,0.000000
6791,9999312,39,2,20.91,1,0,1,0,0.000000


## The final dataframe has 6,793 entries.

- 2,917 persons experienced State 1 (No disease).
- 2,234 persons experienced State 2 (Hypertension) - some of these persons (1,298) never experienced State 1, but began in State 2.
- 650 persons survived State 3 (Cardiovascular disease).
- 992 persons experienced State 4 (Death).

In [34]:
framingham_ms["STATE"].value_counts()

1    2917
2    2234
4     992
3     650
Name: STATE, dtype: int64

In [35]:
framingham_ms.to_csv("Datasets/framingham_ms.csv")

In [30]:
#21 and 22
framingham_ms.iloc[20:22]

,RANDID,AGE,SEX,BMI,CURSMOKE,DIABETES,STATE,DAYS
20,23727,47,2,27.98,0,0,4,5592
21,24721,39,2,22.35,1,0,1,0


In [31]:
framingham_ms[framingham_ms["RANDID"]==344720]

,RANDID,AGE,SEX,BMI,CURSMOKE,DIABETES,STATE,DAYS
234,344720,55,2,24.27,0,0,1,0
235,344720,61,2,23.48,0,0,3,3488
236,344720,61,2,23.48,0,0,2,4334
